In [1]:
import pandas as pd
import numpy as np

from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

from IPython.display import Image
from IPython.core.display import HTML 

In [2]:
from collections import defaultdict

In [3]:
import re

In [4]:
dataset_ok = pd.read_csv("../../dataset_ok.csv")

In [5]:
dataset_ok.head(10)

,text,label
0,"наебалово века, для долбаёбов\n",INSULT
1,вся дума в таком же положении😁\n,NORMAL
2,а в каком месте массовое столкновение? шрайбик...,NORMAL
3,"значит ли это, что контроль за вывозом крупног...",NORMAL
4,вам не нужен щеночек? очень хорошие 🐶🥰\n,NORMAL
5,"он, хоть живой остался??.\n",NORMAL
6,было дело.\n,NORMAL
7,"с хранением нет проблем, или только в холодиль...",NORMAL
8,полностью вас поддерживаю\n,NORMAL
9,этот рецепт не соответствует фото. ооох и наму...,NORMAL


# Упрощённый BPE

In [4]:
with open("../../lenta.txt", encoding="UTF-8") as file:
    corpus_lenta = file.read()

In [36]:
def count_pairs(corpus_joined: list):
    corpus_split = corpus_joined.split(" ")
    counts = defaultdict()
    for i in range(len(corpus_split) - 1):
        if (corpus_split[i], corpus_split[i+1]) in counts:
            counts[corpus_split[i], corpus_split[i+1]] += 1
        else:
            counts[corpus_split[i], corpus_split[i+1]] = 1
    return counts

def bpe_initialize(corpus: str, n_iterations: int, k_pairs: int):
    merged_vocab = set()
    punct = re.compile(r'[ .,:!;+=?""'']')
    corpus = re.sub(punct, "", corpus)
    corpus_symbols = [sym for sym in corpus]
    corpus_joined = " ".join(corpus_symbols)
    for n in range(n_iterations):
        pairs = count_pairs(corpus_joined)
        top_k = sorted(list(pairs.keys()), key=pairs.get, reverse=True)[:k_pairs]
        for pair in top_k:
            new_pair = pair[0] + pair[1]
            corpus_joined = re.sub(f"{pair[0]} {pair[1]}", new_pair, corpus_joined)
            merged_vocab.add(new_pair)
    return merged_vocab

In [39]:
print(list(bpe_initialize(corpus_lenta, 2, 50)))

['ан', 'ма', 'ств', 'ва', 'ам', 'ру', 'со', 'ли', 'ци', 'ных', 'то', 'ей', 'ом', 'ви', 'ча', 'та', 'ож', 'ер', 'вы', 'ил', 'ую', 'ает', 'от', 'ая', 'щи', 'ной', 'ре', 'ор', 'ст', 'ки', 'ед', 'ос', 'ин', 'ав', 'на', 'ны', 'ка', 'ем', 'уд', 'еч', 'пр', 'их', 'им', 'да', 'па', 'уп', 'он', 'ар', 'го', 'ла', 'ог', 'ди', 'ни', 'ку', 'бы', 'об', 'за', 'ми', 'ен', 'ев', 'ес', 'си', 'ет', 'ит', 'ат', 'яв', 'уч', 'по', 'ль', 'од', 'ии', 'ис', 'ля', 'из', 'не', 'ел', 'ле', 'ро', 'ас', 'ри', 'общ', 'ся', 'ра', 'ции', 'уж', 'ко', 'ий', 'ооб', 'те', 'ал', 'ак', 'ски', 'ол', 'во', 'де', 'ск', 'аз', 'ти', 'ов', 'но']


### Комментарий: среди частотных пар много таких, которые соотвествуют распространённым морфемам: "ую", "ся", "ет". Также встречаются фрагменты частотных слов, выходящие за пределы отдельной морфемы: "ции", "ной", "ных", "ает".

In [38]:
print(list(bpe_initialize(corpus_lenta, 5, 20)))

['ан', 'бу', 'хо', 'ма', 'ва', 'ру', 'со', 'ско', 'до', 'под', 'ли', 'ци', 'га', 'ных', 'то', 'ей', 'ви', 'ча', 'при', 'та', 'ер', 'вы', 'ил', 'ком', 'от', 'пу', 'ло', 'ной', 'ре', 'ор', 'ст', 'ки', 'ед', 'ос', 'ть', 'ин', 'ного', 'на', 'ны', 'ду', 'ка', 'ем', 'еч', 'пр', 'их', 'да', 'па', 'бо', 'это', 'го', 'ла', 'ди', 'ни', 'ку', 'бы', 'об', 'за', 'ми', 'ен', 'ев', 'ес', 'си', 'ет', 'ит', 'мо', 'яв', 'по', 'ль', 'ии', 'ис', 'ля', 'из', 'не', 'ов', 'ел', 'ле', 'ро', 'ри', 'общ', 'ся', 'ра', 'уж', 'ко', 'ий', 'ты', 'те', 'ал', 'что', 'ну', 'му', 'са', 'ак', 'лу', 'ски', 'раз', 'во', 'де', 'ти', 'ств', 'но']


### Комментарий: кроме морфем и фрагментов слов, в списке появились отдельные частотные служебные слова: "что", "раз", "это". Самый длинный токен - "ного" с 4 символами.

In [40]:
print(list(bpe_initialize(corpus_lenta, 10, 20)))

['ан', 'гла', 'ского', 'прав', 'ской', 'бу', 'для', 'хо', 'ту', 'ма', 'ва', 'лю', 'еж', 'ру', 'се', 'сво', 'ется', 'его', 'пол', 'со', 'ско', 'до', 'под', 'ли', 'ци', 'га', 'су', 'ном', 'ных', 'жи', 'то', 'Как', 'милли', 'ник', 'ей', 'мож', 'ви', 'пи', 'ча', 'при', 'та', 'гра', 'ная', 'ер', 'ву', 'вы', 'ил', 'дол', 'би', 'ком', 'зи', 'Чеч', 'ает', 'от', 'пу', 'ло', 'ска', 'АН', 'ной', 'ре', 'ор', 'ст', '00', 'зы', 'ки', 'такж', 'На', 'ед', 'бли', 'ба', 'ос', 'ть', 'ин', 'ция', 'ного', 'ря', 'ав', 'на', 'ны', 'ду', 'кой', 'ка', 'ем', 'вя', 'вер', 'еч', 'пр', 'их', 'ме', 'да', 'па', 'ня', 'шени', 'бо', 'фи', 'России', 'ным', 'сообщ', 'Ин', 'это', 'св', 'этом', 'По', 'он', 'фор', 'ар', 'го', 'ла', 'ди', 'чи', 'ни', 'Интер', 'ят', 'гу', 'ку', 'бы', 'мы', 'ный', 'кра', 'об', 'за', 'ИА', 'ми', 'россий', 'ен', 'ную', 'ев', 'сообщает', 'ес', 'си', 'ет', 'года', 'ит', 'ез', 'цен', 'мо', 'яв', 'ствен', 'РИ', 'по', 'пер', 'челов', 'ль', 'ремя', 'од', 'ии', 'ющ', 'ис', 'ке', 'ля', 'из', 'ют', 'не'

### появились длинные отрывки частотных слов: "челов", "президен", "России". Самый длинный токен - "сообщает".

In [48]:
final_vocab = bpe_initialize(corpus_lenta, 10, 50)

In [42]:
text = """С помощью санкций против российских организаций администрация США 
пытается «пнуть и так находящиеся в плохой форме российско-американские отношения», 
заявил пресс-секретарь президента России Дмитрий Песков, передает корреспондент РБК. 
«Это очередной враждебный шаг по отношению к России. Можем только сожалеть, 
что очередная уходящая администрация США предпочитает пнуть и так находящиеся в плохой форме 
российско-американские отношения», — сказал Песков."""

In [46]:
def bpe_tokenize(text: str, vocab: set):
    punct = re.compile(r'[ .,:!;+=?""'']')
    bag_of_tokens = []
    text = re.sub(punct, "", text)
    order = sorted(list(vocab), key=len, reverse=True)
    for token in order:
        if token in text:
            bag_of_tokens.append(token)
    return bag_of_tokens

In [49]:
print(bpe_tokenize(text, final_vocab))

['министра', 'против', 'ерикан', 'россий', 'оссии', 'пресс', 'помощ', 'ские', 'иден', 'ения', 'ется', 'шени', 'чер', 'ает', 'ент', 'пер', 'что', 'ски', 'мер', 'Рос', 'ход', 'фор', 'ной', 'ция', 'орг', 'раж', 'ный', 'со', 'ер', 'ой', 'от', 'щи', 'ка', 'ме', 'чи', 'ес', 'си', 'ит', 'ис', 'ек', 'ся', 'ра', 'ну', 'ак', 'ам', 'га', 'то', 'та', 'ше', 'ож', 'же', 'ло', 'аг', 'ст', 'ию', 'с-', 'ин', 'на', 'их', 'да', 'за', 'ми', 'по', 'од', 'ии', 'из', 'ри', 'ош', 'ал', 'ти', 'ан', 'ью', 'ом', 'ви', 'пе', 'ад', 'ил', 'ая', 'ор', 'ут', 'ть', 'ны', 'пр', 'пы', 'он', 'ен', 'ет', 'ез', 'яв', 'ро', 'ух', 'ий', 'ах', 'ощ', 'ск', 'аз', 'ов', 'се', 'ША', 'ци', 'зи', 'ив', 'ре', 'ки', 'ед', 'ос', 'ем', 'ар', 'ни', 'ох', 'ль', 'оч', 'ле', 'ко', 'са', 'ол', 'де', 'СШ', 'но']


### Комментарий: при применении BPE без предварительной пословной токенизации много информации потенциально теряется

# Реализация TF-IDF

In [6]:
from tokenizers import CharBPETokenizer, Tokenizer

In [7]:
from collections import Counter

In [51]:
dataset_ok['text'].to_csv('corpus_new.txt', index=None)

In [8]:
tok_sub = CharBPETokenizer()
tok_sub.train('corpus_new.txt', vocab_size=6000, min_frequency=10,)

In [9]:
tok_sub.encode(dataset_ok.loc[1, 'text']).tokens

['вся</w>',
 'ду',
 'ма</w>',
 'в</w>',
 'таком</w>',
 'же</w>',
 'поло',
 'жени',
 'и',
 '😁</w>']

In [10]:
from scipy.sparse import lil_matrix

### Подсчёт TF и DF

In [11]:
tokenized_texts = [Counter(tok_sub.encode(dataset_ok.loc[i, 'text']).tokens) for i in range(dataset_ok.shape[0])]

In [13]:
voc = tok_sub.get_vocab()

In [14]:
document_frequency = dict()
tokens = list(voc.keys())
for term in tokens:
    for count in tokenized_texts:
        if term not in count:
            continue
        if term in document_frequency:
            document_frequency[term] += 1
        else:
            document_frequency[term] = 1

In [15]:
document_frequency['в</w>']

16441

### Заполнение матрицы

In [16]:
tf_idf_matrix = lil_matrix((dataset_ok.shape[0], 6000), dtype=np.float32)

In [17]:
Image(url="https://miro.medium.com/max/3604/1*qQgnyPLDIkUmeZKN2_ZWbQ.png",
     width=500, height=500)

### Пользуемся индексами в словаре в качестве индексов колонок

In [18]:
n_docs = dataset_ok.shape[0] + 1 # for computing idf
for index in range(len(tokenized_texts)):
    count = tokenized_texts[index]
    for item in count.keys():
        col_index = voc[item]
        tf = count[item] / sum(count.values())
        idf = n_docs / (document_frequency[item] + 1)
        result = tf * np.log1p(idf)
        tf_idf_matrix[index, col_index] = result

### Обучение классификатора

In [19]:
y = np.array(dataset_ok.label)

In [20]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

In [94]:
clf_log = LogisticRegression(C=1, max_iter=120, n_jobs=3)
cross_val_score(clf_log, tf_idf_matrix, y, scoring="f1_macro", cv=StratifiedKFold(n_splits=5, shuffle=True))

array([0.77072889, 0.77983193, 0.79112207, 0.77533286, 0.76180761])

In [22]:
clf_NB = MultinomialNB()
cross_val_score(clf_NB, tf_idf_matrix, y, scoring="f1_macro", cv=StratifiedKFold(n_splits=5, shuffle=True))

array([0.58090091, 0.56539705, 0.54981246, 0.59286224, 0.59702563])

### Комментарий: поскольку используется метрика "f1_macro", результаты выглядят плохими. Тем не менее логистическая регрессия превосходит по результату классификатор, обученный на семинаре, а наивный байес показывает сходный с ним по качеству результат.

In [22]:
cross_val_score(clf, X, y, scoring="f1_macro", cv=StratifiedKFold(n_splits=5, shuffle=True))

array([0.50043239, 0.50446208, 0.51254333, 0.50846296, 0.51108045])